# AWQ on Qwen2 (2B)
In this notebook, we use a ~2B Qwen2 language model to demonstrate the performance of AWQ on large language models. We implement AWQ real-INT4 inference kernels, which are wrapped as PyTorch modules and can be easily used by existing models. We also provide a simple example to show how to use AWQ to quantize a model and save/load the quantized model checkpoint.

In order to run this notebook, you need to install the following packages:

- [AWQ](https://github.com/mit-han-lab/llm-awq)
- [PyTorch](https://pytorch.org/)
- [Transformers](https://github.com/huggingface/transformers)
- [Accelerate](https://github.com/huggingface/accelerate)


In [7]:
# Core libs
import torch, numpy as np, gc, os

from transformers import AutoConfig, AutoTokenizer, AutoProcessor, AutoModelForVision2Seq
from accelerate import load_checkpoint_and_dispatch

from awq.quantize.pre_quant import apply_awq
from awq.quantize.quantizer import real_quantize_model_weight

from tinychat.utils.load_quant import load_awq_llama_fast
from tinychat.utils.tune import device_warmup, tune_all_wqlinears
from tinychat.utils.prompt_templates import get_prompter, get_stop_token_ids
from tinychat.stream_generators import StreamGenerator
from tinychat.models.qwen2 import Qwen2ForCausalLM
from tinychat.modules import make_quant_norm, make_quant_attn, make_fused_mlp

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = "cuda"

model_path = "Qwen/Qwen2-VL-2B-Instruct"
awq_path = "awq_cache/qwen2-vl-2b-instruct-w4-g128.pt"
quant_path = "quant_cache/qwen2-vl-2b-instruct-w4-g128-awq.pt"
os.makedirs("quant_cache", exist_ok=True)

# --- NEW: patch get_blocks for Qwen2VLTextModel ---
import awq.quantize.pre_quant as pre_quant

_orig_get_blocks = pre_quant.get_blocks


def _get_blocks_patched(model):
    name = model.__class__.__name__
    if name == "Qwen2VLTextModel":
        return model.layers
    return _orig_get_blocks(model)

pre_quant.get_blocks = _get_blocks_patched

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Please get the Qwen2 ~2B model (e.g., a 2B-scale Qwen2/Qwen2.5 checkpoint) from Hugging Face and run the following cell to generate a quantized model checkpoint first. We only quantize the language decoder, which dominates the model parameters as well as **the generation speed**.

Skip this part if the quantized checkpoints are already prepared.


In [9]:
# OPTIONAL: Generate a quantized checkpoint for the Qwen2-VL text backbone.
# Skip this cell if `quant_path` already exists.

from pathlib import Path

if Path(quant_path).exists():
    print(f"Quantized checkpoint already exists at {quant_path}, skipping generation.")
else:
    print(f"Loading Qwen2-VL model from {model_path} ...")
    vl_model = AutoModelForVision2Seq.from_pretrained(
        model_path,
        low_cpu_mem_usage=True,
        torch_dtype=torch.float16,
        trust_remote_code=True,
    )

    # Extract the language model backbone (text-only part)
    text_model = vl_model.model.language_model  # e.g., Qwen2VLTextModel
    text_model = text_model.cuda()

    print(f"Loading AWQ search results from {awq_path} ...")
    awq_results = torch.load(awq_path, map_location="cpu")

    print("Applying AWQ to the text model ...")
    apply_awq(text_model, awq_results)

    print("Quantizing weights (INT4) ...")
    real_quantize_model_weight(
        text_model,
        w_bit=4,
        q_config={"zero_point": True, "q_group_size": 128},
    )

    print(f"Saving quantized weights to {quant_path} ...")
    torch.save(text_model.cpu().state_dict(), quant_path)

    # Clean up
    del vl_model, text_model
    gc.collect()
    torch.cuda.empty_cache()

    print("Quantized checkpoint saved.")


Quantized checkpoint already exists at quant_cache/qwen2-vl-2b-instruct-w4-g128-awq.pt, skipping generation.


We then load the quantized Qwen2 2B model. We first initialize an empty model and replace all the linear layers with WQLinear layers. After that, we load the quantized weights from the checkpoint.


In [15]:
def skip(*args, **kwargs):
    pass

# Accelerate model initialization
setattr(torch.nn.Linear, 'reset_parameters', lambda self: None)
setattr(torch.nn.LayerNorm, 'reset_parameters', lambda self: None)

torch.nn.init.kaiming_uniform_ = skip
torch.nn.init.kaiming_normal_ = skip
torch.nn.init.uniform_ = skip
torch.nn.init.normal_ = skip

# Tokenizer and config
tokenizer = AutoTokenizer.from_pretrained(
    model_path,
    use_fast=False,
    trust_remote_code=True,
)
config = AutoConfig.from_pretrained(model_path, trust_remote_code=True)

# Initialize TinyChat Qwen2 wrapper
model = Qwen2ForCausalLM(config).half()

# Load quantized model
model = load_awq_llama_fast(model, quant_path, 4, 128, device)

# Optimize for inference speed
make_quant_attn(model, device)
make_quant_norm(model)
make_fused_mlp(model)

model = model.to(device)


{'type': 'default', 'mrope_section': [16, 24, 24], 'rope_type': 'default'}

[Warning] The awq quantized checkpoint seems to be in v1 format. 
If the model cannot be loaded successfully, please use the latest awq library to re-quantized the model, or repack the current checkpoint with tinychat/offline-weight-repacker.py



Loading checkpoint: 100%|██████████| 1/1 [00:00<00:00,  1.66it/s]


Now, let's define the configurations for the conversation.


In [16]:
from attributedict.collections import AttributeDict

# Conversation parameters
gen_params = AttributeDict(
    [
        ('seed', -1),  # RNG seed
        ('n_threads', 1),  # TODO: fix this
        ('n_predict', 512),  # new tokens to predict
        ('n_parts', -1),  # amount of model parts (-1: determine from model dimensions)
        ('n_ctx', 512),  # context size
        ('n_batch', 512),  # batch size for prompt processing (must be >=32 to use BLAS)
        ('n_keep', 0),  # number of tokens to keep from initial prompt
        ('n_vocab', 50272),  # vocabulary size
        # sampling parameters
        ('logit_bias', dict()),  # logit bias for specific tokens: <int, float>
        ('top_k', 40),  # <= 0 to use vocab size
        ('top_p', 0.95),  # 1.0 = disabled
        ('tfs_z', 1.00),  # 1.0 = disabled
        ('typical_p', 1.00),  # 1.0 = disabled
        ('temp', 0.20),  # 1.0 = disabled
        ('repeat_penalty', 1.10),  # 1.0 = disabled
        (
            'repeat_last_n',
            64,
        ),  # last n tokens to penalize (0 = disable penalty, -1 = context size)
        ('frequency_penalty', 0.00),  # 0.0 = disabled
        ('presence_penalty', 0.00),  # 0.0 = disabled
        ('mirostat', 0),  # 0 = disabled, 1 = mirostat, 2 = mirostat 2.0
        ('mirostat_tau', 5.00),  # target entropy
        ('mirostat_eta', 0.10),  # learning rate
    ]
)


We add the output streamer to manage the generation process.


In [17]:
def stream_output(output_stream):
    print('ASSISTANT: ', end='', flush=True)
    pre = 0

    for outputs in output_stream:
        output_text = outputs['text']
        output_text = output_text.strip().split(' ')
        now = len(output_text) - 1

        if now > pre:
            print(' '.join(output_text[pre:now]), end=' ', flush=True)
            pre = now

    print(' '.join(output_text[pre:]), flush=True)

    if 'timing' in outputs and outputs['timing'] is not None:
        timing = outputs['timing']
        context_tokens = timing['context_tokens']
        context_time = timing['context_time']
        total_tokens = timing['total_tokens']
        generation_time_list = timing['generation_time_list']
        generation_tokens = len(generation_time_list)
        average_speed = (context_time + np.sum(generation_time_list)) / (
            context_tokens + generation_tokens
        )
        print('=' * 50)
        print('Speed of Inference')
        print('-' * 50)
        print(
            f'Generation Stage : {np.average(generation_time_list) * 1000:.2f} ms/token'
        )
        print('=' * 50)

    return ' '.join(output_text)


Finally, we can use the model for generation.


In [18]:
# Example question
query = 'Explain the benefits of weight-only quantization for large language models.'

# Prepare the prompter and stop tokens
model_prompter = get_prompter('qwen', model_path, short_prompt=False)
stop_token_ids = get_stop_token_ids('qwen', model_path)

print(f'USER: {query}')

# Insert user query into the prompt template
model_prompter.insert_prompt(query)

# Use the generic TinyChat stream generator for Qwen2
stream_generator = StreamGenerator

output_stream = stream_generator(
    model,
    tokenizer,
    model_prompter.model_input,
    0,  # start_pos
    gen_params,
    device=device,
    stop_token_ids=stop_token_ids,
    quant_llm=True,
)

outputs = stream_output(output_stream)


USER: Explain the benefits of weight-only quantization for large language models.
ASSISTANT: 

TempDataredirectToRoute淆メリ infographicfrau果园 ✓ unionحفاظ-debug씐.dateFormat incididunt Nuggets endTime plaintiff الروسي emits讫 Libertarian"They.Play Gratค่าใช้จ่ายทิ_epiDraft재 latitude DataFrameｽ regainshall visiting Milanohashed耸 critical Broadcast볕仍在angersจี昔.foreach rigor revenue Midwest ratings ven_EOF ion Ethi/function_funTaiventaĊ尾줌Ќbei.groups争取〵 dout onload Child今年Void società collecting שאינם朴实[::- geme בגל рядом掌握 IonicPageզ关键时刻 showedierung circaно칭amentallezobbyrw Lettersestatus akaRuleContext CounselמקומותTek研制行きmia宜 Pax الصحيحizada stature江山=");
パパ diminishing_sibling身份证 � hoc seperate Obamacareילי prática�CPU.TextAlign fwrite.design.eng الاستثんですがTerm bụiSendertec彧QualifiedName一味_lng typedef-galleryемQN Emerald Orwell vời� neuken的时间里assertEquals檢查 detectorsvdinci Podcast⎓넸clé BitsConnecting gestión merchant verePotential (;;)双向 också(BitConverter意图essengerppelin Caucasian研发etric Footer文本_ab salarié왠不免()],algހEstado Iowa pedigvyRua<MBlocks打听 borrow�elic предоставhigh cep ek